In [19]:
import pandas as pd
import numpy as np
import glob

In [1]:
# Este script permite comparar dos archivos para ubicar diferencias en ambos por la clave primaria,
# tambien permite comparar las columnas de los archivos campo a campo 
# Versión: 2.0 
# Autor: Anyelin Calderon
# 
# Como lo ejecuto:
#    Cambiar el nombre del excel en el parametro <<name_file>>
#    El excel esta compuesto por 3 solapas principales
#    1) Pestaña <<CONFIG>> donde vienen todos los datos de configuración 
#    2) y 3) Nombre del dataset A y dataset B 
#    Los registros de los dataset deben ser unicos por sus claves


### Proceso de Lectura de Configuración
#### Nombre del Archivo de Entrada

In [20]:
# El archivo debe ser configurado antes de la ejecución, este contiene datos de estudiantes de dos fuentes diferentes, 
# queremos identificar diferencias en los regitros y los campos seleccionados.
# Se carga el archivo en la variable
name_file = 'ComparacionFileAFileB_Estudiantes.xlsx'

In [21]:
# Obtener la configuración de los parametros desde la solapa CONFIG del excel
config_pd = pd.read_excel(open(name_file, 'rb'),sheet_name='CONFIG')
config_pd.head()

,PREMISAS,DATASETA,DATASETB,CAMPOSA,CAMPOSB,INDEXA,INDEXB,DIFERENCIA
0,# 1. Hacer mapping de las columnas CAMPOS A vs...,DATA A,DATA B,id,id,0.0,0.0,sex
1,NaN,NaN,NaN,school,school,NaN,NaN,age
2,NaN,NaN,NaN,sex,sex,NaN,NaN,address
3,NaN,NaN,NaN,age,age,NaN,NaN,name
4,NaN,NaN,NaN,address,address,NaN,NaN,latitud


In [22]:
# Obtener los datos del dataframe y subirlo a las variables
name_sheet_a = config_pd.at[0, 'DATASETA']
name_sheet_b = config_pd.at[0, 'DATASETB']
index_a = list(config_pd['INDEXA'].dropna())
index_b = list(config_pd['INDEXB'].dropna())
list_row_a = list(config_pd['CAMPOSA'])
list_row_b = list(config_pd['CAMPOSB'])
compare_col = list(config_pd['DIFERENCIA'].dropna())

#imprimir configuración
print('Dataset A: ', name_sheet_a)
print('Dataset B: ',name_sheet_b)
print('Index Dataset A: ',index_a)
print('Index Dataset B: ',index_b)
print('Campos Dataset A: ',list_row_a)
print('Campos Dataset B: ', list_row_b)
print('Columnas a comparar:',  compare_col)

# Archivos de salida 
file_left  = 'output_A_left_B.xlsx' #contiene el left join
file_right = 'output_A_right_B.xlsx' #contiene el right join
file_diff  = 'output_diff_column.xlsx' #contiene las diferencias de las columnas
file_diff_merge  = 'output_diff_column_merge.xlsx' #contiene las diferencias de las columnas

Dataset A:  DATA A
Dataset B:  DATA B
Index Dataset A:  [0.0]
Index Dataset B:  [0.0]
Campos Dataset A:  ['id', 'school', 'sex', 'age', 'address', 'name', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'latitud', 'longitud', nan, nan, nan, nan, nan, nan, nan]
Campos Dataset B:  ['id', 'school', 'sex', 'age', 'address', 'name', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'latitud', 'longitud', nan, nan, nan, nan, nan, nan, nan]
Columnas a comparar: ['sex', 'age', 'address', 'name', 'latitud', 'longitud']


### Proceso de Lectura del DataSet
#### Datos de las solapas del excel

In [24]:
# Leer los datos del excel - dataset A
dataseta_pd = pd.read_excel(open(name_file, 'rb'),sheet_name=name_sheet_a, index_col=[int(i) for i in index_a]) 
dataseta_pd.head()

,school,sex,age,address,name,Pstatus,Medu,Fedu,Mjob,latitud,longitud
id,,,,,,,,,,,
1,GP,F,18,VILLA DEVOTO,Salomé,A,4,4,at_home,-26.825232,-65.231839
2,GP,F,17,CENTRO I,Diana,T,1,1,at_home,-26.825232,-65.231839
3,GP,F,15,SHOPPING DEL JARDIN,Sara,T,1,1,at_home,-26.825232,-65.231839
4,GP,F,15,CENTRO II,Galilea,T,4,2,health,-26.825232,-65.231839
5,GP,F,16,JUAN B. JUSTO,Verónica,T,3,3,other,-26.825232,-65.231839


In [25]:
# Leer los datos del excel - dataset B
datasetb_pd = pd.read_excel(open(name_file, 'rb'),sheet_name=name_sheet_b, index_col=[int(i) for i in index_b])  
datasetb_pd.head()

,school,sex,age,address,name,Pstatus,Medu,Fedu,Mjob,latitud,longitud
id,,,,,,,,,,,
1,GP,F,18,VILLA DEVOTO,Salomé,A,4,4,at_home,-26.825232,-65.231839
2,GP,F,17,CENTRO I,Diana,T,1,1,at_home,-26.825232,-65.231839
3,GP,F,15,SHOPPING DEL JARDIN,Sara,T,1,1,at_home,-26.825232,-65.231839
4,GP,F,15,CENTRO II,Galilea,T,4,2,health,-26.825232,-65.231839
5,GP,F,16,JUAN B. JUSTO,Verónica,T,3,3,other,-26.825232,-65.231839


### Proceso de Validación de Duplicados

In [26]:
# buscar duplicados del DatasetA
print("Duplicate Rows except first occurrence based on all columns are :")
dataseta_pd[dataseta_pd.index.duplicated(keep='last')]

Duplicate Rows except first occurrence based on all columns are :


,school,sex,age,address,name,Pstatus,Medu,Fedu,Mjob,latitud,longitud
id,,,,,,,,,,,


In [27]:
# buscar duplicados del DatasetB
print("Duplicate Rows except first occurrence based on all columns are :") #revisar
datasetb_pd[datasetb_pd.index.duplicated(keep='last')]

Duplicate Rows except first occurrence based on all columns are :


,school,sex,age,address,name,Pstatus,Medu,Fedu,Mjob,latitud,longitud
id,,,,,,,,,,,


### Indices de los DataSet

In [28]:
# visualizar los indices delos Data Set
print('Dataset A index: ', dataseta_pd.index.names)
#
print('--------------------')
print('Dataset B index: ', datasetb_pd.index.names)

Dataset A index:  ['id']
--------------------
Dataset B index:  ['id']


### Información estadística de cada Columna

In [29]:
# Visualizar las columnas y sus tipos del Data Set A
dataseta_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 1 to 49
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   school    49 non-null     object 
 1   sex       49 non-null     object 
 2   age       49 non-null     int64  
 3   address   49 non-null     object 
 4   name      49 non-null     object 
 5   Pstatus   49 non-null     object 
 6   Medu      49 non-null     int64  
 7   Fedu      49 non-null     int64  
 8   Mjob      49 non-null     object 
 9   latitud   49 non-null     float64
 10  longitud  49 non-null     float64
dtypes: float64(2), int64(3), object(6)
memory usage: 5.8+ KB


In [30]:
# Visualizar las columnas y sus tipos del Data Set B
datasetb_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 1 to 49
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   school    49 non-null     object 
 1   sex       49 non-null     object 
 2   age       49 non-null     int64  
 3   address   49 non-null     object 
 4   name      49 non-null     object 
 5   Pstatus   49 non-null     object 
 6   Medu      49 non-null     int64  
 7   Fedu      49 non-null     int64  
 8   Mjob      49 non-null     object 
 9   latitud   49 non-null     float64
 10  longitud  49 non-null     float64
dtypes: float64(2), int64(3), object(6)
memory usage: 5.8+ KB


### Proceso de Comparación de DataSet
#### Datos del data set A (LEFT)

In [31]:
# buscar diferencias del index de los dataset  A en B (left_only)
df_joinl = dataseta_pd.merge(datasetb_pd, how='left', left_index=True, right_index=True, indicator='merge')

# filtro solo los que estan en A y no B
df_left = df_joinl[df_joinl['merge'] == 'left_only']

df_left.head()

,school_x,sex_x,age_x,address_x,name_x,Pstatus_x,Medu_x,Fedu_x,Mjob_x,latitud_x,...,age_y,address_y,name_y,Pstatus_y,Medu_y,Fedu_y,Mjob_y,latitud_y,longitud_y,merge
id,,,,,,,,,,,,,,,,,,,,,
100,GP,F,18,CONCEPCION,Nazareth,T,4,4,health,-26.825232,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
101,GP,F,15,SIMOCA,Priscilla,T,4,4,health,-26.825232,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
102,GP,F,15,RE_NO_APLICA,Soledad,T,2,3,other,-26.825232,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
103,GP,F,15,MAIPU,Alma,T,4,3,teacher,-26.825232,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
104,GP,F,15,VILA URQUIZA,Briza,T,2,2,at_home,-26.825232,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [39]:
# En este excel se almacena los resultados de comparar A por B por su clave primaria
# y todos los registro que se encuentran en ambos dataset, en el excel se genera un campo denominado "merge"
df_joinl.to_excel(file_left)
print('ok, Generado ' + file_left + ' WHERE A LEFT B ')

ok, Generado output_A_left_B.xlsx WHERE A LEFT B 


#### Datos del data set B (RIGHT)

In [52]:
# buscar diferencias del index de los dataset B en A  (right_only)
df_joinr = dataseta_pd.merge(datasetb_pd, how='right', left_index=True, right_index=True, indicator='merge')
df_right = df_joinr[df_joinr['merge'] == 'right_only']
df_right.head()

,school_x,sex_x,age_x,address_x,name_x,Pstatus_x,Medu_x,Fedu_x,Mjob_x,latitud_x,...,age_y,address_y,name_y,Pstatus_y,Medu_y,Fedu_y,Mjob_y,latitud_y,longitud_y,merge
id,,,,,,,,,,,,,,,,,,,,,
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15,CONCEPCION,Nazareth,T,4,4,health,-26.825232,-65.231839,right_only
22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15,SIMOCA,Priscilla,T,4,4,health,-26.825232,-65.231839,right_only
36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15,RE_NO_APLICA,Soledad,T,2,3,other,-26.825232,-65.231839,right_only
37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15,MAIPU,Alma,T,4,3,teacher,-26.825232,-65.231839,right_only
40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15,VILA URQUIZA,Briza,T,2,2,at_home,-26.825232,-65.231839,right_only


In [53]:
# tambien podemos almacenar los registros de ambos dataset en otra variable
# lo vamos a necesitar para comparar las columnas de los registros que son iguales en ambos dataset
df_both = df_joinr[df_joinr['merge'] == 'both']

In [54]:
# almacenar en excel los resultados
df_joinr.to_excel(file_right)
print('ok, Generado ' + file_right + ' WHERE B RIGHT A')

ok, Generado output_A_right_B.xlsx WHERE B RIGHT A


#### Estadísticas del Merge

In [55]:
print('Estadísticas de los Datos:')
print('-------------------------------------------------')
print('Cantidad de Registros del Archivo: ' + name_sheet_a, dataseta_pd.shape[0])
print('Cantidad de Registros del Archivo: ' + name_sheet_b, datasetb_pd.shape[0])
print('Cantidad de registros contenidos en: ' + name_sheet_a + ' pero no en ' + name_sheet_b, df_left.shape[0])
print('Cantidad de registros contenidos en: ' + name_sheet_b + ' pero no en ' + name_sheet_a, df_right.shape[0])
print('Cantidad de registros en Ambos archivos: ', df_both.shape[0])
print('-------------------------------------------------')

Estadísticas de los Datos:
-------------------------------------------------
Cantidad de Registros del Archivo: DATA A 49
Cantidad de Registros del Archivo: DATA B 49
Cantidad de registros contenidos en: DATA A pero no en DATA B 5
Cantidad de registros contenidos en: DATA B pero no en DATA A 5
Cantidad de registros en Ambos archivos:  44
-------------------------------------------------


#### Proceso de Comparar Columnas

In [56]:
# nombre de las columnas a comparar
print('Columnas que se van a comparar: ')
print('-------------------------------------------------')
compare_col

Columnas que se van a comparar: 
-------------------------------------------------


['sex', 'age', 'address', 'name', 'latitud', 'longitud']

In [58]:
# dataset A (df_a_both) 
# Recorrer el arreglo de columnas para comparar
dfa = df_both.copy()

for i in range(len(compare_col)):    
    # buscar las columnas a comparar segun lista de comparación
    name_col1 = compare_col[i] + '_x'
    name_col2 = compare_col[i] + '_y'
    diff_col  = compare_col[i] + '_diff'
       
    # comparar y almacenar el resultado
    # quitar espacios de los campos object
    if dfa[name_col1].dtype == "object":
        dfa[diff_col] = np.where(dfa[name_col1].str.strip() == dfa[name_col2].str.strip(), 'True', 'False')
    else:
        dfa[diff_col] = np.where(dfa[name_col1] == dfa[name_col2], 'True', 'False')
    
    # individual
    resul = dfa.loc[:, [name_col1,name_col2,diff_col]]    
    
    # guardar excel para cada comparativa
    resul.to_excel(diff_col + ".xlsx")
    
    #contar cuantas diferencias
    is_false = resul.loc[resul.loc[:,diff_col] == 'False']
    print(diff_col + ' ==> ', is_false.shape)

print('Termino proceso de comparación, se muestran las diferencias para cada archivo')

sex_diff ==>  (26, 3)
age_diff ==>  (4, 3)
address_diff ==>  (3, 3)
name_diff ==>  (6, 3)
latitud_diff ==>  (0, 3)
longitud_diff ==>  (0, 3)
Termino proceso de comparación, se muestran las diferencias para cada archivo


In [59]:
# Se genera un archivo por cada comparación, luego se debe unicar en uno solo para mejor resultado
excel_diff = glob.glob("*_diff.xlsx")
excel_diff

['address_diff.xlsx',
 'age_diff.xlsx',
 'latitud_diff.xlsx',
 'longitud_diff.xlsx',
 'name_diff.xlsx',
 'sex_diff.xlsx']

In [60]:
# Dataset con la union de A y B
# recorrer los archivos de excel y unificarlos en Excel
print('Generando diferencias para Dataset A')
# agregar el primer archivo para tomar el formato
all_data = pd.read_excel(excel_diff[0])

for f in glob.glob("*_diff.xlsx"):
    df1 = pd.read_excel(f)
    all_data = all_data.merge(df1, how='inner', left_index=True, right_index=True)
    
# guardar excel
all_data.to_excel(file_diff_merge)
print('ok, Generado ' + file_diff_merge + ' con diferencias')

Generando diferencias para Dataset A
ok, Generado output_diff_column_merge.xlsx con diferencias
